In [4]:
import os
import fnmatch
from random import shuffle
import json
from config import config_io,original_datasets, pan21_format,processed
from pathlib import Path
import json
import pandas as pd
from nltk import sent_tokenize

In [49]:
selected_dataset = 'valid_2019'
data_path = pan21_format[selected_dataset]
print(data_path)
os.listdir(data_path)
base = [file.split(".")[0] for file in os.listdir(data_path) if "txt" in file]
truth_files = ["truth-"+file+".json" for file in base]
problem_files = [file+".txt" for file in base] 
#file_list = [data_path+'/'+file for file in temp_file_list]

/home/sukanya/PhD/CLEF/converted_data/2019/validation


In [20]:
def verify_file_exists(file_list,data_path):
    for file in file_list:
        if not Path(data_path+"/"+file).exists():
            print("Error! Could not find file: ",data_path+file)

In [21]:
verify_file_exists(truth_files,data_path)
verify_file_exists(problem_files,data_path)

In [22]:
'''def get_w1_w2(sentence_list, window_size):
    w1_list_temp = []
    w2_list_temp = []
    for i in range(1, window_size+1):
        w1_list_temp.append(sentence_list[0:i])
        print(sentence_list[0:i])
    for i in range(1, len(sentence_list)-window_size):
        w1_list_temp.append(sentence_list[i:i+window_size])
        print(sentence_list[i:i+window_size])
    for i in range(1, len(sentence_list)):
        w2_list_temp.append(sentence_list[i:i+window_size])
        print(sentence_list[i:i+window_size])
    if len(w1_list_temp)!=len(w2_list_temp):
        print("Both lists not equal")
    w1_list = []
    w2_list = []
    for w1,w2 in zip(w1_list_temp, w2_list_temp):
        w1_concat = " ".join(w1)
        w2_concat = " ".join(w2)
        print("w1:", w1_concat, ", w2:", w2_concat)'''

'def get_w1_w2(sentence_list, window_size):\n    w1_list_temp = []\n    w2_list_temp = []\n    for i in range(1, window_size+1):\n        w1_list_temp.append(sentence_list[0:i])\n        print(sentence_list[0:i])\n    for i in range(1, len(sentence_list)-window_size):\n        w1_list_temp.append(sentence_list[i:i+window_size])\n        print(sentence_list[i:i+window_size])\n    for i in range(1, len(sentence_list)):\n        w2_list_temp.append(sentence_list[i:i+window_size])\n        print(sentence_list[i:i+window_size])\n    if len(w1_list_temp)!=len(w2_list_temp):\n        print("Both lists not equal")\n    w1_list = []\n    w2_list = []\n    for w1,w2 in zip(w1_list_temp, w2_list_temp):\n        w1_concat = " ".join(w1)\n        w2_concat = " ".join(w2)\n        print("w1:", w1_concat, ", w2:", w2_concat)'

In [44]:
def get_w1_w2(problem_file, sentence_list, window_size, paragraph_authors):
    w1_list_temp = []
    w2_list_temp = []
    a1_list = [] #author of w1
    a2_list = [] #author of w2
    print("sentences number:", len(sentence_list))
    print("para len:",len(paragraph_authors))
    if len(sentence_list)<=window_size:
        print(f"Error!Problem is skipped as length of sentences {len(sentence_list)} is less than {window_size} for {problem_file}")
        return None, None, None, None, None
    for i in range(1, window_size+1):
        print("i=", i)
        w1_list_temp.append(sentence_list[0:i])
        #print(sentence_list[0:i])
        if(i-1)<len(paragraph_authors):
            a1_list.append(paragraph_authors[i-1])
        else:
            a1_list.append(paragraph_authors[-1])
    
    for i in range(1, len(sentence_list)-window_size):
        w1_list_temp.append(sentence_list[i:i+window_size])
        #print(sentence_list[i:i+window_size])
        if (i+window_size-1)<len(paragraph_authors):
            a1_list.append(paragraph_authors[i+window_size-1])
        else:
            a1_list.append(paragraph_authors[-1])
    
    for i in range(1, len(sentence_list)):
        w2_list_temp.append(sentence_list[i:i+window_size])
        #print(sentence_list[i:i+window_size])
        if i<len(paragraph_authors):
            a2_list.append(paragraph_authors[i]) #-1
        else:
            a2_list.append(paragraph_authors[-1])
        
    if len(w1_list_temp)!=len(w2_list_temp):
        print("Both lists not equal")
    w1_list = []
    w2_list = []
    for w1,w2 in zip(w1_list_temp, w2_list_temp):
        w1_concat = " ".join(w1)
        w2_concat = " ".join(w2)
        w1_list.append(w1_concat)
        w2_list.append(w2_concat)
        #print("w1:", w1_concat, ", w2:", w2_concat)
    problem_list = [problem_file]*len(w1_list)
    if len(a1_list)!=len(a2_list):
        print("Error!len not equal", problem_file)
        print("len a1: ",len(a1_list))
        print("len a2", len(a2_list))
        return None, None, None, None, None
    else:
    #df = pd.DataFrame({"problem": problem_list,"author_1": a1_list, "author_2": a2_list, "window_1": w1_list,"window_2": w2_list})
        return problem_list, a1_list, a2_list, w1_list, w2_list

In [30]:
def convertPAN_to_siamese_format(text,truth_data, problem_file,window_size):
    paragraph_authors = truth_data['paragraph-authors']
    sentences = sent_tokenize(text)
    '''if len(sentences)!= len(paragraph_authors):
        print(f"Error! Number of sentences {len(non_zero_len_paragraphs)} does not equal paragraph authors {len(paragraph_authors)}! God help you now!")
        print(paragraph_authors)
        for para in sentences:
            print(f"\n len: {len(para)} {para}")
            return None
    else:'''
    return get_w1_w2(problem_file, sentences, window_size, paragraph_authors)

In [50]:
problem_list=[]
author1_list=[]
author2_list=[]
window1_list=[]
window2_list=[]
window_size = 3
for i in range(0,len(truth_files)): #range(0,len(truth_files))
    truth_file = truth_files[i]
    problem_file = problem_files[i]
    print((truth_file, problem_file))
    with open(data_path+"/"+problem_file) as f:
        text = f.read()
    print("text len:", len(text))
    with open(data_path+"/"+truth_file) as json_file:
        data = json.load(json_file)
    p_list, a1_list, a2_list, w1_list, w2_list = convertPAN_to_siamese_format(text,data, problem_file,window_size)
    if p_list is not None and a1_list is not None and a2_list is not None and w1_list is not None and w2_list is not None:
        for p in p_list:
            problem_list.append(p)
        for a1 in a1_list:
            author1_list.append(a1)
        for a2 in a2_list:
            author2_list.append(a2)
        for w1 in w1_list:
            window1_list.append(w1)
        for w2 in w2_list:
            window2_list.append(w2)
    

('truth-problem-631.json', 'problem-631.txt')
text len: 3834
sentences number: 53
para len: 1
i= 1
i= 2
i= 3
('truth-problem-1068.json', 'problem-1068.txt')
text len: 4143
sentences number: 36
para len: 1
i= 1
i= 2
i= 3
('truth-problem-347.json', 'problem-347.txt')
text len: 10220
sentences number: 77
para len: 77
i= 1
i= 2
i= 3
('truth-problem-378.json', 'problem-378.txt')
text len: 4171
sentences number: 32
para len: 1
i= 1
i= 2
i= 3
('truth-problem-793.json', 'problem-793.txt')
text len: 8337
sentences number: 75
para len: 1
i= 1
i= 2
i= 3
('truth-problem-287.json', 'problem-287.txt')
text len: 3210
sentences number: 36
para len: 36
i= 1
i= 2
i= 3
('truth-problem-771.json', 'problem-771.txt')
text len: 11632
sentences number: 99
para len: 99
i= 1
i= 2
i= 3
('truth-problem-1156.json', 'problem-1156.txt')
text len: 7760
sentences number: 70
para len: 1
i= 1
i= 2
i= 3
('truth-problem-719.json', 'problem-719.txt')
text len: 9215
sentences number: 81
para len: 1
i= 1
i= 2
i= 3
('truth-pr

i= 3
('truth-problem-850.json', 'problem-850.txt')
text len: 9836
sentences number: 112
para len: 112
i= 1
i= 2
i= 3
('truth-problem-1020.json', 'problem-1020.txt')
text len: 9572
sentences number: 82
para len: 82
i= 1
i= 2
i= 3
('truth-problem-1050.json', 'problem-1050.txt')
text len: 13500
sentences number: 93
para len: 93
i= 1
i= 2
i= 3
('truth-problem-110.json', 'problem-110.txt')
text len: 7878
sentences number: 76
para len: 1
i= 1
i= 2
i= 3
('truth-problem-788.json', 'problem-788.txt')
text len: 8700
sentences number: 90
para len: 90
i= 1
i= 2
i= 3
('truth-problem-550.json', 'problem-550.txt')
text len: 3837
sentences number: 31
para len: 1
i= 1
i= 2
i= 3
('truth-problem-1177.json', 'problem-1177.txt')
text len: 8913
sentences number: 72
para len: 1
i= 1
i= 2
i= 3
('truth-problem-1169.json', 'problem-1169.txt')
text len: 6457
sentences number: 62
para len: 62
i= 1
i= 2
i= 3
('truth-problem-713.json', 'problem-713.txt')
text len: 6100
sentences number: 65
para len: 1
i= 1
i= 2
i= 

text len: 16997
sentences number: 143
para len: 143
i= 1
i= 2
i= 3
('truth-problem-82.json', 'problem-82.txt')
text len: 1882
sentences number: 15
para len: 1
i= 1
i= 2
i= 3
('truth-problem-9.json', 'problem-9.txt')
text len: 6104
sentences number: 54
para len: 54
i= 1
i= 2
i= 3
('truth-problem-727.json', 'problem-727.txt')
text len: 6267
sentences number: 64
para len: 1
i= 1
i= 2
i= 3
('truth-problem-988.json', 'problem-988.txt')
text len: 13963
sentences number: 177
para len: 177
i= 1
i= 2
i= 3
('truth-problem-692.json', 'problem-692.txt')
text len: 3701
sentences number: 30
para len: 30
i= 1
i= 2
i= 3
('truth-problem-1131.json', 'problem-1131.txt')
text len: 15364
sentences number: 139
para len: 139
i= 1
i= 2
i= 3
('truth-problem-824.json', 'problem-824.txt')
text len: 2700
sentences number: 25
para len: 1
i= 1
i= 2
i= 3
('truth-problem-327.json', 'problem-327.txt')
text len: 5141
sentences number: 67
para len: 67
i= 1
i= 2
i= 3
('truth-problem-404.json', 'problem-404.txt')
text len

para len: 126
i= 1
i= 2
i= 3
('truth-problem-129.json', 'problem-129.txt')
text len: 10843
sentences number: 101
para len: 101
i= 1
i= 2
i= 3
('truth-problem-120.json', 'problem-120.txt')
text len: 3329
sentences number: 28
para len: 1
i= 1
i= 2
i= 3
('truth-problem-421.json', 'problem-421.txt')
text len: 8670
sentences number: 57
para len: 57
i= 1
i= 2
i= 3
('truth-problem-1031.json', 'problem-1031.txt')
text len: 3768
sentences number: 24
para len: 1
i= 1
i= 2
i= 3
('truth-problem-815.json', 'problem-815.txt')
text len: 10051
sentences number: 81
para len: 81
i= 1
i= 2
i= 3
('truth-problem-330.json', 'problem-330.txt')
text len: 6670
sentences number: 45
para len: 1
i= 1
i= 2
i= 3
('truth-problem-498.json', 'problem-498.txt')
text len: 8470
sentences number: 70
para len: 1
i= 1
i= 2
i= 3
('truth-problem-1187.json', 'problem-1187.txt')
text len: 14996
sentences number: 129
para len: 129
i= 1
i= 2
i= 3
('truth-problem-1115.json', 'problem-1115.txt')
text len: 2280
sentences number: 20


i= 1
i= 2
i= 3
('truth-problem-336.json', 'problem-336.txt')
text len: 8745
sentences number: 86
para len: 86
i= 1
i= 2
i= 3
('truth-problem-729.json', 'problem-729.txt')
text len: 6610
sentences number: 68
para len: 1
i= 1
i= 2
i= 3
('truth-problem-560.json', 'problem-560.txt')
text len: 4938
sentences number: 57
para len: 1
i= 1
i= 2
i= 3
('truth-problem-845.json', 'problem-845.txt')
text len: 9337
sentences number: 88
para len: 88
i= 1
i= 2
i= 3
('truth-problem-1086.json', 'problem-1086.txt')
text len: 5496
sentences number: 53
para len: 1
i= 1
i= 2
i= 3
('truth-problem-39.json', 'problem-39.txt')
text len: 7568
sentences number: 69
para len: 69
i= 1
i= 2
i= 3
('truth-problem-99.json', 'problem-99.txt')
text len: 3817
sentences number: 28
para len: 1
i= 1
i= 2
i= 3
('truth-problem-1255.json', 'problem-1255.txt')
text len: 8150
sentences number: 68
para len: 68
i= 1
i= 2
i= 3
('truth-problem-645.json', 'problem-645.txt')
text len: 7284
sentences number: 62
para len: 62
i= 1
i= 2
i= 3

...

sentences number: 36
para len: 36
i= 1
i= 2
i= 3
('truth-problem-193.json', 'problem-193.txt')
text len: 4634
sentences number: 54
para len: 1
i= 1
i= 2
i= 3
('truth-problem-15.json', 'problem-15.txt')
text len: 5244
sentences number: 56
para len: 1
i= 1
i= 2
i= 3
('truth-problem-121.json', 'problem-121.txt')
text len: 13825
sentences number: 105
para len: 105
i= 1
i= 2
i= 3
('truth-problem-134.json', 'problem-134.txt')
text len: 7418
sentences number: 71
para len: 71
i= 1
i= 2
i= 3
('truth-problem-57.json', 'problem-57.txt')
text len: 5601
sentences number: 45
para len: 45
i= 1
i= 2
i= 3
('truth-problem-1104.json', 'problem-1104.txt')
text len: 10650
sentences number: 80
para len: 80
i= 1
i= 2
i= 3
('truth-problem-592.json', 'problem-592.txt')
text len: 2731
sentences number: 9
para len: 1
i= 1
i= 2
i= 3
('truth-problem-807.json', 'problem-807.txt')
text len: 8353
sentences number: 83
para len: 1
i= 1
i= 2
i= 3
('truth-problem-1163.json', 'problem-1163.txt')
text len: 10071
sentences 

i= 3
('truth-problem-402.json', 'problem-402.txt')
text len: 2830
sentences number: 20
para len: 1
i= 1
i= 2
i= 3
('truth-problem-549.json', 'problem-549.txt')
text len: 6090
sentences number: 58
para len: 1
i= 1
i= 2
i= 3
('truth-problem-1160.json', 'problem-1160.txt')
text len: 5476
sentences number: 34
para len: 1
i= 1
i= 2
i= 3
('truth-problem-385.json', 'problem-385.txt')
text len: 6009
sentences number: 43
para len: 1
i= 1
i= 2
i= 3
('truth-problem-499.json', 'problem-499.txt')
text len: 6786
sentences number: 60
para len: 60
i= 1
i= 2
i= 3
('truth-problem-744.json', 'problem-744.txt')
text len: 5669
sentences number: 57
para len: 1
i= 1
i= 2
i= 3
('truth-problem-389.json', 'problem-389.txt')
text len: 9923
sentences number: 92
para len: 1
i= 1
i= 2
i= 3
('truth-problem-1173.json', 'problem-1173.txt')
text len: 10635
sentences number: 108
para len: 108
i= 1
i= 2
i= 3
('truth-problem-1076.json', 'problem-1076.txt')
text len: 8236
sentences number: 64
para len: 64
i= 1
i= 2
i= 3
('

text len: 2935
sentences number: 24
para len: 1
i= 1
i= 2
i= 3
('truth-problem-1190.json', 'problem-1190.txt')
text len: 7012
sentences number: 51
para len: 1
i= 1
i= 2
i= 3
('truth-problem-102.json', 'problem-102.txt')
text len: 4800
sentences number: 36
para len: 36
i= 1
i= 2
i= 3
('truth-problem-333.json', 'problem-333.txt')
text len: 8905
sentences number: 61
para len: 1
i= 1
i= 2
i= 3
('truth-problem-632.json', 'problem-632.txt')
text len: 8652
sentences number: 90
para len: 90
i= 1
i= 2
i= 3
('truth-problem-643.json', 'problem-643.txt')
text len: 9445
sentences number: 127
para len: 127
i= 1
i= 2
i= 3
('truth-problem-842.json', 'problem-842.txt')
text len: 2318
sentences number: 25
para len: 1
i= 1
i= 2
i= 3
('truth-problem-160.json', 'problem-160.txt')
text len: 12496
sentences number: 124
para len: 124
i= 1
i= 2
i= 3
('truth-problem-649.json', 'problem-649.txt')
text len: 8316
sentences number: 55
para len: 1
i= 1
i= 2
i= 3
('truth-problem-1136.json', 'problem-1136.txt')
text l

sentences number: 116
para len: 116
i= 1
i= 2
i= 3
('truth-problem-912.json', 'problem-912.txt')
text len: 8121
sentences number: 78
para len: 78
i= 1
i= 2
i= 3
('truth-problem-646.json', 'problem-646.txt')
text len: 8793
sentences number: 109
para len: 109
i= 1
i= 2
i= 3
('truth-problem-864.json', 'problem-864.txt')
text len: 6842
sentences number: 60
para len: 1
i= 1
i= 2
i= 3
('truth-problem-140.json', 'problem-140.txt')
text len: 2601
sentences number: 23
para len: 1
i= 1
i= 2
i= 3
('truth-problem-781.json', 'problem-781.txt')
text len: 11682
sentences number: 82
para len: 82
i= 1
i= 2
i= 3
('truth-problem-574.json', 'problem-574.txt')
text len: 7133
sentences number: 75
para len: 1
i= 1
i= 2
i= 3
('truth-problem-255.json', 'problem-255.txt')
text len: 6965
sentences number: 60
para len: 1
i= 1
i= 2
i= 3
('truth-problem-1002.json', 'problem-1002.txt')
text len: 9028
sentences number: 79
para len: 1
i= 1
i= 2
i= 3
('truth-problem-353.json', 'problem-353.txt')
text len: 5479
sentence

sentences number: 53
para len: 1
i= 1
i= 2
i= 3
('truth-problem-431.json', 'problem-431.txt')
text len: 2503
sentences number: 25
para len: 1
i= 1
i= 2
i= 3
('truth-problem-154.json', 'problem-154.txt')
text len: 6270
sentences number: 79
para len: 1
i= 1
i= 2
i= 3
('truth-problem-66.json', 'problem-66.txt')
text len: 4220
sentences number: 46
para len: 46
i= 1
i= 2
i= 3
('truth-problem-825.json', 'problem-825.txt')
text len: 8589
sentences number: 74
para len: 1
i= 1
i= 2
i= 3
('truth-problem-983.json', 'problem-983.txt')
text len: 13799
sentences number: 120
para len: 120
i= 1
i= 2
i= 3
('truth-problem-658.json', 'problem-658.txt')
text len: 14838
sentences number: 120
para len: 120
i= 1
i= 2
i= 3
('truth-problem-874.json', 'problem-874.txt')
text len: 4844
sentences number: 45
para len: 1
i= 1
i= 2
i= 3
('truth-problem-517.json', 'problem-517.txt')
text len: 13262
sentences number: 112
para len: 112
i= 1
i= 2
i= 3
('truth-problem-852.json', 'problem-852.txt')
text len: 3126
sentence

In [51]:
print("problem_list",len(problem_list))
print("author1_list",len(author1_list))
print("author2_list",len(author2_list))
print("window1_list",len(window1_list))
print("window2_list",len(window2_list))

problem_list 82042
author1_list 82042
author2_list 82042
window1_list 82042
window2_list 82042


In [52]:
df = pd.DataFrame({"problem": problem_list,
                   "author_1": author1_list,
                   "author_2": author2_list,
                   "window_1": window1_list,
                   "window_2": window2_list})
    

In [53]:
df.to_csv(processed[selected_dataset])

In [ ]:
def convertPAN_to_siamese_format(text,truth_data, problem_file):
    a1_list = []
    a2_list = []
    p1_list = []
    p2_list = []
    p_list = []
    paragraph_authors = truth_data['paragraph-authors']
    sentences = nltk.sent_tokenize(text)
    #paragraphs = text.split("\n")
    #non_zero_len_paragraphs = [para for para in paragraphs if len(para)>0] # problem-601 creates an empty paragraph causing the problem. so had to remove empty paragraphs
    if len(sentences)!= len(paragraph_authors):
        print(f"Error! Number of sentences {len(non_zero_len_paragraphs)} does not equal paragraph authors {len(paragraph_authors)}! God help you now!")
        print(paragraph_authors)
        for para in sentences:
            print(f"\n len: {len(para)} {para}")
            return None, None, None, None
    else:
        for i in range(0, len(paragraph_authors)):
            a1 = paragraph_authors[i]
            if i+1<len(paragraph_authors) and (i+1)< len(paragraphs):
                a2 = paragraph_authors[i+1]
                p1 = non_zero_len_paragraphs[i]
                p2 = non_zero_len_paragraphs[i+1]
                a1_list.append(a1)
                p1_list.append(p1)
                a2_list.append(a2)
                p2_list.append(p2)
                p_list.append(problem_file)
    return p_list, a1_list, a2_list, p1_list, p2_list

Approach
- first split text according to chnages position
- for each author excerpt check if same author cases can be generated..i.e split at paragraph change.

In [5]:
a1_list = []
a2_list = []
p1_list = []
p2_list = []
p_list = []
for i in range(0,len(truth_files)):
    truth_file = truth_files[i]
    problem_file = problem_files[i]
    print((truth_file, problem_file))
    with open(data_path+problem_file) as f:
        text = f.read()
    print("len:", len(text))
    with open(data_path+truth_file) as json_file:
        data = json.load(json_file)
    structure = data['structure']
    changes = data['switches']
    start = 0
    excerpts = []
    author = structure[0]
    for i in range(0, len(changes)):
        end = changes[i]
        excerpts.append((author, text[start:end]))
        start = end
        author = structure[i+1]
    excerpts.append((author, text[start:]))
    count = 0
    #print("excerpts", excerpts)
    #for i,j in excerpts:
    #    count = count+len(j)
    #    print("\n\nauthor:",i,"position: ",count)#"para:\n\n"+j
    para_author_list = []
    for author,excerpt in excerpts:
        temp = excerpt.split("\n\n")
        paragraphs = [para for para in temp if len(para)>100]
        for para in paragraphs:
            para_author_list.append((author,para))
    print("para_author_list", len(para_author_list))
    if len(para_author_list)==1:
        author,para_text = para_author_list[0]
        print("para_text:", para_text)
        if len(para_text)>400:
            a1 = author
            p1 = para_text[:int(len(para_text)/2)]
            a2 = author
            p2 = para_text[int(len(para_text)/2):]
            print(p1,p2)
            a1_list.append(a1)
            p1_list.append(p1)
            a2_list.append(a2)
            p2_list.append(p2)
            p_list.append(problem_file)
        
    for i in range(1, len(para_author_list)):
        a1 = para_author_list[i-1][0]
        p1 = para_author_list[i-1][1]
        a2 = para_author_list[i][0]
        p2 = para_author_list[i][1]
        a1_list.append(a1)
        p1_list.append(p1)
        a2_list.append(a2)
        p2_list.append(p2)
        p_list.append(problem_file)
        print(a1,a2)
data = {"problem":p_list, "author_1":a1_list, "author_2":a2_list, "para1_text": p1_list, "para2_text":p2_list}
df = pd.DataFrame(data)

('problem-631.truth', 'problem-631.txt')
len: 3834
para_author_list 9
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
('problem-1068.truth', 'problem-1068.txt')
len: 4143
para_author_list 1
para_text: 
I recommend $URL$ on the proof of Mertens by Mark Villarino.  He shows that in fact the opposite inequality (less than) holds with A=4, and also shows the error term from the original proof (with an additional 2/(n ln n) term) and calculation of B.  I believe there are some oscillation results by Diamond and Pintz which would show that the posted inequality (greater than) never holds for all n sufficiently large for any positive A .
Gerhard "Now On The Right Side" Paseman, 2017.05.14. 
I am putting the final touches on a writeup (approximately 12 pages) which gives $f(n) \lt 1.607 n/\log n$ and a little more.  Those who wish a copy may request one by email. 
The general idea (with much detail omitted) is this: for every integer $m$ find where possible $n\lt m$ with $f(m)n\log m \lt f(n)m

len: 6267
para_author_list 1
para_text: Yes, exactly. The system itself doesn't presume that assets have any particular value. There is no problem with worthless assets existing in the system because there is no way to get such an asset to someone who didn't agree to take it.
Like most other payment systems, Ripple only controls what goes on inside itself. It can't compel people to do things outside of the system.
The risk-averse way to protect yourself from this is to trust only reliable gateways. The more adventurous way to do it is to trust only people whose Ripple network has value in such a way that they can't betray your trust without giving up their Ripple network. This is the basis of community credit, which really isn't ready for prime time just yet. 
G is short for "giga" (billions), just like M is short for "mega" (millions). Congratulations on a very lucky find that, sadly, won't pay you any more than any other share. The same way you do with other currencies -- you pay thr

len: 2242
para_author_list 3
A1 A1
A1 A1
('problem-366.truth', 'problem-366.txt')
len: 2329
para_author_list 3
A1 A1
A1 A1
('problem-32.truth', 'problem-32.txt')
len: 8305
para_author_list 8
A1 A2
A2 A1
A1 A2
A2 A1
A1 A2
A2 A1
A1 A2
('problem-115.truth', 'problem-115.txt')
len: 5329
para_author_list 2
A1 A1
('problem-1230.truth', 'problem-1230.txt')
len: 8809
para_author_list 14
A1 A1
A1 A1
A1 A2
A2 A2
A2 A1
A1 A1
A1 A2
A2 A2
A2 A1
A1 A1
A1 A1
A1 A2
A2 A2
('problem-654.truth', 'problem-654.txt')
len: 16593
para_author_list 25
A1 A2
A2 A2
A2 A2
A2 A1
A1 A3
A3 A3
A3 A2
A2 A2
A2 A1
A1 A3
A3 A3
A3 A3
A3 A4
A4 A4
A4 A4
A4 A4
A4 A2
A2 A2
A2 A3
A3 A3
A3 A3
A3 A3
A3 A4
A4 A4
('problem-527.truth', 'problem-527.txt')
len: 3455
para_author_list 9
A1 A2
A2 A1
A1 A2
A2 A2
A2 A1
A1 A2
A2 A1
A1 A2
('problem-1044.truth', 'problem-1044.txt')
len: 10788
para_author_list 14
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A2
A2 A2
A2 A2
A2 A2
A2 A2
A2 A2
A2 A2
('problem-336.truth', 'problem-336.txt')
len: 8745
par

len: 22725
para_author_list 13
A1 A5
A5 A5
A5 A3
A3 A5
A5 A5
A5 A1
A1 A2
A2 A3
A3 A3
A3 A2
A2 A4
A4 A5
('problem-610.truth', 'problem-610.txt')
len: 8684
para_author_list 1
para_text: 
Take a look at , e.g.: Now let's try that again... No need to set any attributes doing it this way. Prepend  if you want all factors from all numbers in submitted list grouped, e.g.: Use: If entries are integer,  s/b faster, but will fail for non-integers... The results are quite sparse (e.g., for 10 variables and power of 12, only about 0.00000082% of the array is non-zero), so this should get you somewhere with memory pressure. It is quite a bit faster in my tests on the turdbook, YMMV.
If the order of results is not relevant (that is, you only care about the coefficient values), drop the ordering part, that will save time and some memory. The results with ordering applied follow the permutations of the partitions of the degree of the result (that is, the order that say CoefficientRules poops it out), 

len: 9593
para_author_list 25
A1 A1
A1 A2
A2 A2
A2 A2
A2 A1
A1 A1
A1 A1
A1 A2
A2 A2
A2 A1
A1 A1
A1 A2
A2 A2
A2 A2
A2 A1
A1 A2
A2 A2
A2 A1
A1 A1
A1 A1
A1 A1
A1 A2
A2 A2
A2 A2
('problem-741.truth', 'problem-741.txt')
len: 4472
para_author_list 1
para_text: 
Yes, that is exactly what the client does. It uses heuristics to do this, solving a subset/sum or knapsack problem.
It uses a multi-pass approach, first trying to use only coins with at least six confirmations. In the next two passes it relaxes these requirements. Within each pass, it tries to minimize the number of transaction outputs claimed and then the amount of change returned.
Note that all coins in your wallet are considered. In the accounting model that the Bitcoin client uses, specific coins do not belong to specific accounts.
If you want to check the code yourself, search for  in . 
I think it's conceivable that this could happen. But under realistic cases, I don't think it actually will. If you don't mine on the longest blo

len: 6964
para_author_list 19
A1 A4
A4 A4
A4 A2
A2 A2
A2 A4
A4 A4
A4 A4
A4 A2
A2 A2
A2 A4
A4 A4
A4 A4
A4 A2
A2 A2
A2 A3
A3 A3
A3 A4
A4 A4
('problem-1066.truth', 'problem-1066.txt')
len: 9689
para_author_list 6
A1 A2
A2 A4
A4 A2
A2 A3
A3 A4
('problem-948.truth', 'problem-948.txt')
len: 2588
para_author_list 1
para_text: 
Go to 'Tasks' List Settings > 'Name' Lookup column
Under Additional column settings, you might have 'Add a column to show each of these additional fields' , there choose the 'Person Involved' and Save.
Now modify the view, under Filter > Choose 'Name:Person Involved' column and the save the filter as it is equal to [Me] This will provide you the count of MS Office files uploaded to all the SharePoint Site Collections in the Content DBs. 
I have a SharePoint Project. When Visual Studio project has finished building, it should generate SharePoint Solution Package(WSP) automatically.
How can I achieve this automation, using visual studio ? 
To increase the width of a parti

len: 7408
para_author_list 7
A1 A4
A4 A1
A1 A2
A2 A1
A1 A3
A3 A4
('problem-398.truth', 'problem-398.txt')
len: 8417
para_author_list 13
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
('problem-369.truth', 'problem-369.txt')
len: 12467
para_author_list 18
A1 A2
A2 A2
A2 A4
A4 A4
A4 A2
A2 A2
A2 A4
A4 A4
A4 A4
A4 A1
A1 A3
A3 A1
A1 A4
A4 A4
A4 A1
A1 A1
A1 A1
('problem-932.truth', 'problem-932.txt')
len: 5839
para_author_list 1
para_text: I've been doing some experimenting with a MicroSD card formatted as ExFAT and have come across something that perplexes me.
It seems that my new Android phone mounts the MicroSD card as , so for example rather than  as I'm used to, it's currently .
Because that's fairly arbitrary, I thought I'd change it to something easier to remember, like  (my initials in the second half).  I used tune2fs to specify a new UUID, and was able to confirm the change by opening the raw disk in a hex editor.
I actually discovered the UUID is stored at

para_author_list 6
A1 A2
A2 A1
A1 A3
A3 A2
A2 A4
('problem-83.truth', 'problem-83.txt')
len: 9181
para_author_list 16
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
A1 A1
('problem-1155.truth', 'problem-1155.txt')
len: 6507
para_author_list 5
A1 A1
A1 A1
A1 A1
A1 A1
('problem-1200.truth', 'problem-1200.txt')
len: 2599
para_author_list 1
para_text: 
Generally speaking, condensation problems only manifest when moving your gear from/to a nice, warm interior to/from the cold outdoors. Once outside, once your gear has adjusted to the temperature, the concern is not nearly as great. The 7D is a professional grade camera, and as such is fully weather resistant. So long as you use professional grade lenses that are also weather sealed, and do not change your lenses while outdoors, you shouldn't have any problems with condensation building up inside your camera. You might still have to wait for any external condensation to dissipate before you can take any p

In [120]:
df.head()


,problem,author_1,author_2,para1_text,para2_text
0,problem-1543.txt,A1,A1,Given an alphabet $\Sigma$ of size $k$ and two...,"\nThe paper ""Algorithmic Meta Theorems for Cir..."
1,problem-631.txt,A1,A3,"\nInstead of an actual GPS device, I would rec...","In the end, a seat post pump probably has the ..."
2,problem-631.txt,A3,A2,"In the end, a seat post pump probably has the ...",\nThanks for the help. Finally what I needed t...
3,problem-631.txt,A2,A1,\nThanks for the help. Finally what I needed t...,"\nIt might sound weird, but if you're finding ..."
4,problem-631.txt,A1,A2,"\nIt might sound weird, but if you're finding ...",This is a bit weird question. I bought a Speci...


In [6]:
df.to_csv("/home/sukanya/PhD/Datasets/PAN SCD/pan19-style-change-detection/processed/test.csv",index=False)